In [4]:
from tkinter import *
from math import inf as infinity
from random import choice

# Constants for the players
HUMAN, COMP = -1, +1

# Initial game state
board = [[0, 0, 0] for _ in range(3)]
game_is_over = False
human_choice, comp_choice, current_player = '', '', ''

def evaluate(state):
    return +1 if wins(state, COMP) else -1 if wins(state, HUMAN) else 0

def wins(state, player):
    win_states = (
        [[state[i][j] for j in range(3)] for i in range(3)] +
        [[state[j][i] for j in range(3)] for i in range(3)] +
        [[state[i][i] for i in range(3)], [state[i][2 - i] for i in range(3)]]
    )
    return [player, player, player] in win_states

def game_over(state):
    return wins(state, HUMAN) or wins(state, COMP)

def empty_cells(state):
    return [[x, y] for x, row in enumerate(state) for y, cell in enumerate(row) if cell == 0]

def valid_move(x, y):
    return [x, y] in empty_cells(board)

def set_move(x, y, player):
    if valid_move(x, y):
        board[x][y] = player
        return True
    return False

def minimax(state, depth, player):
    if depth == 0 or game_over(state):
        return [-1, -1, evaluate(state)]
    best = [-1, -1, -infinity if player == COMP else +infinity]
    for x, y in empty_cells(state):
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[:2] = [x, y]
        best = max(best, score, key=lambda x: x[2]) if player == COMP else min(best, score, key=lambda x: x[2])
    return best

def ai_turn():
    global game_is_over, current_player
    if game_is_over or len(empty_cells(board)) == 0:
        return
    if len(empty_cells(board)) == 9:
        move = [choice([0, 1, 2]), choice([0, 1, 2])]
    else:
        move = minimax(board, len(empty_cells(board)), COMP)
    set_move(*move[:2], COMP)
    render()
    check_game_over()
    current_player = human_choice

def human_turn(x, y):
    if valid_move(x, y):
        set_move(x, y, HUMAN)
        render()
        check_game_over()
        if not game_is_over:
            current_player = comp_choice
            ai_turn()

def render():
    for i in range(3):
        for j in range(3):
            buttons[i][j].config(text="O" if board[i][j] == HUMAN else "X" if board[i][j] == COMP else " ", fg="blue" if board[i][j] == HUMAN else "red")

def check_game_over():
    global game_is_over
    if game_over(board):
        result_label.config(text=f"Game Over! {'Human' if wins(board, HUMAN) else 'Computer'} wins!")
        game_is_over = True
    elif len(empty_cells(board)) == 0:
        result_label.config(text="Game Over! It's a Draw!")
        game_is_over = True

def on_click(row, col):
    if not game_is_over and current_player == human_choice:
        human_turn(row, col)

def reset_game():
    global board, game_is_over, current_player
    board = [[0] * 3 for _ in range(3)]
    game_is_over = False
    current_player = ''
    for row in buttons:
        for button in row:
            button.config(text=" ")
    result_label.config(text="")
    show_choice_frame()

def start_game():
    global human_choice, comp_choice, current_player
    human_choice = human_choice_var.get().upper()
    comp_choice = 'O' if human_choice == 'X' else 'X'
    current_player = first_turn_var.get()
    if current_player == 'Computer':
        current_player = comp_choice
        ai_turn()
    else:
        current_player = human_choice
    choice_frame.pack_forget()
    game_frame.pack()

def show_choice_frame():
    choice_frame.pack()

root = Tk()
root.title("Tic-Tac-Toe")

choice_frame = Frame(root)
Label(choice_frame, text="Choose X or O:", font=('Arial', 20)).pack(pady=10)
human_choice_var = StringVar(value='X')
Radiobutton(choice_frame, text="X", variable=human_choice_var, value='X', font=('Arial', 20)).pack()
Radiobutton(choice_frame, text="O", variable=human_choice_var, value='O', font=('Arial', 20)).pack()

Label(choice_frame, text="Do you want to start first?", font=('Arial', 20)).pack(pady=10)
first_turn_var = StringVar(value='Human')
Radiobutton(choice_frame, text="Yes", variable=first_turn_var, value='Human', font=('Arial', 20)).pack()
Radiobutton(choice_frame, text="No", variable=first_turn_var, value='Computer', font=('Arial', 20)).pack()

Button(choice_frame, text="Start Game", font=('Arial', 20), command=start_game).pack(pady=20)

game_frame = Frame(root)
Label(game_frame, text="Tic-Tac-Toe", font=('Arial', 30)).grid(row=0, column=0, columnspan=3)

buttons = [[None for _ in range(3)] for _ in range(3)]
for i in range(3):
    for j in range(3):
        buttons[i][j] = Button(game_frame, text=" ", font=('Arial', 60), width=5, height=2, command=lambda i=i, j=j: on_click(i, j))
        buttons[i][j].grid(row=i+1, column=j)

result_label = Label(game_frame, text="", font=('Arial', 20))
result_label.grid(row=4, column=0, columnspan=3)

Button(game_frame, text="Reset", font=('Arial', 20), command=reset_game).grid(row=5, column=0, columnspan=3)

show_choice_frame()

root.mainloop()
